In [ ]:
import time
import os

import numpy as np
import cv2
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import psutil

import pyautogui
from pywinauto import Application

In [2]:
certificate_password = os.getenv('CERTIFICATE_PASSWROD')
driver_path = os.getenv("DRIVER_PATH", 'C:/dev/Chrome/chromedriver')
download_file_path = os.getenv("DOWNLOAD_FILE_PATH", 'C:/dev/files')
date_from = os.getenv("DATE_FROM", "20180901")
date_to = os.getenv("DATE_TO", "20190309")
account_info_path = os.getenv("ACCOUNT_INFO_PATH", 'bank/계좌내역_통합1.xlsx')

In [ ]:
options = Options()
# chrome user agent를 지정해주기
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36")

#     "download.default_directory" : // String which specifies where to download files to by default.
#     "download.prompt_for_download" : // Boolean which specifies whether we should ask the user if we should download // a file (true) or just download it automatically.
#     "download.directory_upgrade" : // Boolean that records if the download directory was changed by an // upgrade a unsafe location to a safe location.
#     "safebrowsing.enabled" : 

options.add_experimental_option('prefs', {
    "download.default_directory": download_file_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

driver = webdriver.Chrome(driver_path, options=options)
driver.implicitly_wait(5)
driver.get('https://obank.kbstar.com/quics?page=C055068&QSL=F#loading')
cookies = driver.get_cookies()

driver.maximize_window()
button_for_login = driver.find_element_by_xpath("//button[@class='btn'][@title='새창']")
button_for_login.click()

# 공인인증서를 위한 sleep 3초 
time.sleep(4)
pyautogui.screenshot('kb/current.png')

### 공인인증서 로그인 (pywinauto)

In [ ]:

process_id = 0
for proc in psutil.process_iter():
    if "delfino" in proc.name():
        process_id = proc.pid
        break
        
delfino = Application(backend='uia').connect(process=process_id)
window = delfino.top_window()

cancel_button = window.child_window(title="취소", control_type="Button")
ok_button = window.child_window(title="확인", control_type="Button")
Pane_object = window.Pane3

key_board = window.GroupBox.Pane
Edit = key_board.Pane2.Edit
Edit.select()
pyautogui.typewrite(certificate_password)
time.sleep(2)
ok_button.click()

### 엑셀파일다운로드

### <a href="#none" class="btn-sub1" onclick="goTranHis('93048550456','2012.03.23','1', '99', '010100200028'); return false;">조회</a>

### <select name="계좌번호" id="계좌번호" class="selbox" onclick="" onchange="">
	<option value="">-선택-</option><option value="93048550456" selected="selected">93048550456 : KB 樂 Star통장-저축예금 </option>

</select>

### <button type="button" class="btn-com c2" onclick="uf_GoSubmit();return false">조회</button>

In [ ]:
from selenium.webdriver.support.ui import Select

click_for_enquiry = driver.find_element_by_xpath("//a[contains(@onclick, 'goTranHis')][text()='조회']")
click_for_enquiry.click()

select_account_list = driver.find_element_by_xpath("//select[@id='계좌번호']")
select_account_list = Select(select_account_list)

account_number_list = []
for account_info in select_account_list.options:
    account_number = str(account_info.get_attribute('value'))
    
    print(account_info)
    print(account_number)
    if account_number == '':
        continue
    else:
        account_number_list.append(account_number)

    account_info.click()
    button_inquiry = driver.find_element_by_xpath("//button[contains(@onclick, 'uf_GoSubmit')][text()='조회']")
    input_inquiry_duration_from = driver.find_element_by_xpath("//input[@id='SJDATE']")
    input_inquiry_duration_to = driver.find_element_by_xpath("//input[@id='JRDATE']")
    input_inquiry_duration_from.send_keys(date_from)
    input_inquiry_duration_to.send_keys(date_to)
    button_excel_download = driver.find_element_by_xpath("//button[@title='엑셀 저장']")
    button_excel_download.click()
    

In [ ]:
# <a href="#none" class="btn-sub1" onclick="goTranHis('93048550456','2012.03.23','1', '99', '010100200028'); return false;">조회</a>

pandas.read_html(url)은 HTML 페이지에 포함된 TABLE들을 DataFrame 의 리스트로 반환한다.

### account_number_list를 하나씩 반환하면서 append한다.

In [ ]:
# index로 구분하고 
# index=False를 해서 to_excel로는 

In [10]:
from openpyxl import load_workbook
import glob
import os
import pandas as pd
from win32com.client import Dispatch

file_list = glob.glob(os.path.join(download_file_path, f"KB_거래내역조회*.xls"))

for file_path in file_list:
    file_path = xls_to_xlsx(file_path)
    
    wb = load_workbook(file_path)
    ws = wb.worksheets[0]
    account_number = ws.cell(row=1, column=1).value
    wb.close()
    
    new_dataframe = pd.read_excel(file_path,skiprows=4, header=0)
    new_dataframe = new_dataframe.set_index([new_dataframe["거래일시"].astype('str').str.strip(), new_dataframe["보낸분/받는분"].astype('str').str.strip()])
    new_dataframe.to_excel("C:/dev/files/취합.xlsx")
    
    sheet_name = ""
    isUpdated = False
    with pd.ExcelFile(account_info_path) as excel_object:
        for tmp_sheet in excel_object.sheet_names:
            if account_number in tmp_sheet:
                sheet_name = tmp_sheet
                isUpdated = True
                break
            else:
                sheet_name = account_number + "_국민은행"
                isUpdated = False
                
        
        if isUpdated:
            old_dataframe = excel_object.parse(sheet_name=sheet_name, header=0)
            old_dataframe = old_dataframe.set_index([old_dataframe["거래일시"].astype('str').str.strip(), old_dataframe["보낸분/받는분"].astype('str').str.strip()])
            new_dataframe = new_dataframe.append(old_dataframe)
            new_dataframe = new_dataframe[~new_dataframe.index.duplicated(keep='first')]
            wb = load_workbook(account_info_path)
            ws = wb[sheet_name]
            wb.remove(ws)
            wb.save(account_info_path)
    
    
    new_dataframe = new_dataframe[~new_dataframe["거래일시"].isnull()]  
    # web에서 다운로드 받은 것을 삭제한다.
    os.remove(file_path)
    
    
    with pd.ExcelWriter(account_info_path, engine='openpyxl') as excel_writer:
        wb = load_workbook(account_info_path)
        excel_writer.book = wb
        new_dataframe.to_excel(excel_writer, sheet_name=sheet_name, index=False)
        excel_writer.save()
    
    

                    거래일시       적요          보낸분/받는분 송금메모      출금액      입금액  \
0    2019.03.04 19:42:31    CD 공동       AMARJARGAL  NaN        0    68238   
1    2019.03.04 06:07:59    스마트입금   JUMDAANBAASANJ  NaN        0    30000   
2    2019.03.03 16:17:01     체크카드       세븐일레븐탑골공원점  NaN     1300        0   
3    2019.03.03 14:12:48    FBS입금              김은영  NaN        0    61500   
4    2019.03.03 13:36:19     전자금융              고승윤  NaN        0    61500   
5    2019.03.03 13:19:42     체크카드        (주)피앤티컴퍼니  NaN     6000        0   
6    2019.03.03 09:29:59     직불출금       비자해외승인대금출금  NaN    15281        0   
7    2019.03.03 03:16:22     체크카드     서울택시_개인영세_KS  NaN     9800        0   
8    2019.03.02 17:13:51     체크카드         위드미고덕으뜸점  NaN     4400        0   
9    2019.03.01 04:02:42     체크카드      선물하기(쿠)_카카오  NaN     9800        0   
10   2019.02.28 12:30:13   FBS 출금          kakao머니  NaN    10000        0   
11   2019.02.27 20:51:10     체크카드      선물하기(쿠)_카카오  NaN    36000        0   

In [4]:
def xls_to_xlsx(xls_file_path):
    
    # replace 함수를 통해서 *.xls 에서 *.xlsx로 새로운 문자열을 만들어줍니다.
    xlsx_file_path = xls_file_path.replace(".xls", ".xlsx").replace("/", "\\")
    excel = Dispatch('Excel.Application')
    wb = excel.Workbooks.Open(xls_file_path)
    
    # fileformat 51은 xlsx에 대한 magic number입니다. 
    wb.SaveAs(xlsx_file_path, FileFormat=51)
    wb.close
    excel.Quit()
    
#     os.remove(xls_file_path)
    return xlsx_file_path
